**Это моё решение задачи в качестве сабмишена для финального проекта специализации 
на курсере по введение в машинное обучение от вышки и Яндекса**

Для начала просто извлечём данные: сделаем это при помощи ранее выбранных признаков. Поскольку цели показать  большой балл на лидерборде у меня нет, то обойдёмся этим неполным набором признаков.

In [42]:
import pandas as pd
train_data = pd.read_csv('./features.csv', index_col='match_id')
test_data = pd.read_csv('./features_test.csv', index_col='match_id')
train_data.head(5)

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


Предлагаю посмотреть более детально какие признаки мы имеем

In [43]:
features = []
for label in train_data.columns:
    features.append(label)
print(features)

['start_time', 'lobby_type', 'r1_hero', 'r1_level', 'r1_xp', 'r1_gold', 'r1_lh', 'r1_kills', 'r1_deaths', 'r1_items', 'r2_hero', 'r2_level', 'r2_xp', 'r2_gold', 'r2_lh', 'r2_kills', 'r2_deaths', 'r2_items', 'r3_hero', 'r3_level', 'r3_xp', 'r3_gold', 'r3_lh', 'r3_kills', 'r3_deaths', 'r3_items', 'r4_hero', 'r4_level', 'r4_xp', 'r4_gold', 'r4_lh', 'r4_kills', 'r4_deaths', 'r4_items', 'r5_hero', 'r5_level', 'r5_xp', 'r5_gold', 'r5_lh', 'r5_kills', 'r5_deaths', 'r5_items', 'd1_hero', 'd1_level', 'd1_xp', 'd1_gold', 'd1_lh', 'd1_kills', 'd1_deaths', 'd1_items', 'd2_hero', 'd2_level', 'd2_xp', 'd2_gold', 'd2_lh', 'd2_kills', 'd2_deaths', 'd2_items', 'd3_hero', 'd3_level', 'd3_xp', 'd3_gold', 'd3_lh', 'd3_kills', 'd3_deaths', 'd3_items', 'd4_hero', 'd4_level', 'd4_xp', 'd4_gold', 'd4_lh', 'd4_kills', 'd4_deaths', 'd4_items', 'd5_hero', 'd5_level', 'd5_xp', 'd5_gold', 'd5_lh', 'd5_kills', 'd5_deaths', 'd5_items', 'first_blood_time', 'first_blood_team', 'first_blood_player1', 'first_blood_playe

Отлично! Теперь мы видим перед глазами набор признаков, которые у нас есть. Сразу же хочется понять как их можно объединить... С одной стороны мы видим всевозможные статистики по каждому из героев. Много ли от этого толка? Не уверен, потому что игра командная...
<br>На самом деле надо немного объянить типичную ситауцию в доте - есть мидер, керри, саппорты и оффлейнер. Как правило саппорты не имеют ни денег, ни опыта, покупают вещи на команду(варды, хилки, телепорты и так далее). Мидеры стоят 1 на 1, имеют много опыта и их задача на первые 5 минут обыграть вражеского мидера, получить много опыта и задоминировать линию и как правило, мидеры задают динамику боя - быстрее всех получают сильную способность(ультимейт) и гангают другие лайны, убивая врагов. Задача керри - фармить деньги, убивая крипов, чтобы в конце боя убить всех остальных - как правило такие герои критичны к крипстату. Ну и наконец есть оффлейнеры - врываторы, инициаторы -  они стоят против 2-3 героев... Их задача получить как можно больше опыта, при этом желательно не умирая.
<br>Этакие правило дотера - ботинки на первой минуте - плохо, нет куры - плохо, варды не стоят - саппорты раки. Фид врагов тоже понятен.
<br>Понятно, что в целом, из-за командности игры можно ввести признак, который будет показывать общие показатели команды... Хотя он и будет линейно зависим от остальных.
<br>Теперь что касается результатов матча - у нас есть статистики по зданиям и победитель встречи... Целевой переменно тут конечно является "radiant_win"

Теперь давайте уберём лишние признаки

In [44]:
#запишем нашу целевую переменную
y_train = train_data['radiant_win']
#запомним лишние данные
towers_dict = ['duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 
               'barracks_status_dire', 'radiant_win']
towers_df = train_data[towers_dict]

#Да, мы записали эти данные отдельно, но ещё не убрали их из трейна
train_df = train_data.drop(towers_dict, axis=1)
train_df.head(5)

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


Теперь всё хорошо, но по факту наши признаки нужно обрабатывать и обрабатывать. Да, большинство из них числовые, но не все, да и значения  некоторых остутствуют

In [45]:
#Первым делом посчитаем в каких столбцах у нас сколько значений
data_nulls = {}
print(len(y_train))
for label in train_df.columns:
    if train_df[label].count() < 97230:
        data_nulls[label]=[train_df[label].count(), test_data[label].count()]
print(data_nulls) 

97230
{'radiant_flying_courier_time': [69751, 12292], 'first_blood_time': [77677, 13625], 'first_blood_player1': [77677, 13625], 'first_blood_player2': [53243, 9411], 'first_blood_team': [77677, 13625], 'dire_courier_time': [96554, 17047], 'radiant_courier_time': [96538, 17050], 'radiant_first_ward_time': [95394, 16847], 'radiant_bottle_time': [81539, 14282], 'dire_bottle_time': [81087, 14335], 'dire_first_ward_time': [95404, 16914], 'dire_flying_courier_time': [71132, 12653]}


Давайте возьмём 2 признака с наибольшим числом пропусков и попробуем про них что-то сказать:
<br>1)<b>"first_blood_player2"</b> - первый килл был сделан без ассистов 
<br>2)<b>"radiant_flying_courier_time"</b> - кура не нужна =D Учитывая что доступ к ней с 3+ минуты

In [46]:
#Уберём пропуски!
train_df = train_df.fillna(0).astype('int')
test_df = test_data.fillna(0).astype('int')

Теперь самое весёлое - время <b>Обучения</b>!

In [6]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
kf = KFold(n_splits=5, shuffle=True)
tree_amount = [10, 20, 30]
for n_tree in tqdm(tree_amount):
    clf = GradientBoostingClassifier(n_estimators=n_tree)
    print(cross_val_score(clf, train_df, y_train, scoring='roc_auc', cv=kf).mean())
    

  0%|          | 0/3 [00:00<?, ?it/s] 33%|███▎      | 1/3 [00:33<01:06, 33.20s/it]

0.664647480884


 67%|██████▋   | 2/3 [01:31<00:40, 40.75s/it]

0.682183356598


100%|██████████| 3/3 [02:55<00:00, 53.65s/it]

0.689532166304
CPU times: user 2min 53s, sys: 1.29 s, total: 2min 54s
Wall time: 2min 55s


Итак, что делать, если число деревьев растёт? Ответ как мне кажется тут довольно прост. Как мы видим из результатов обучения над прошлым множеством, то можно сделать вывод, что максимум пока не достигнут, соотвественно можно продолжать тюнить нашу модель. Поскольку мы используем кросс-валидацию, то переобучения быть не должно. Вопрос о том, как ускорить наш алгоритм? Мне кажется, что одна минута - не такой долгий срок. Рост времени от кол-ва деревьев является линейным.
<br>Ответы на вопрос про ускорение уже есть внизу - можно уменьшить обучающую выборку - деревья малочувствительны к размеру, можно уменьшить глубину, поскольку в градиентном бустинге в отличие от случайного леса нам важна ширина. Есть стохастический градиентный бустинг. Там случайные подвыборки... Наверное можно посмотреть на фичи и выкинуть признаки, которые слабо коррелируют с целевой переменной.

<b>Часть вторая. Логистическая регрессия</b>

Итак, у нас есть уже предобработанный сет. Теперь его надо нормализовать и применить к нему логистическую регрессию. Давайте это и сделаем!
<br>Чтобы подобрать параметр C можно конечно воспользоваться гридом, но поскольку от нас требуют повторить прошлую задачу, то давайте сделаем это вручную

In [7]:
%%time
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np

X_train = StandardScaler().fit_transform(train_df)
C_space = np.logspace(-3, 2, 10)

for c in tqdm(C_space):
    clf = LogisticRegression(C=c)
    print(c, cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=kf).mean())

  0%|          | 0/10 [00:00<?, ?it/s] 10%|█         | 1/10 [00:08<01:19,  8.82s/it]

0.001 0.716127417478


 20%|██        | 2/10 [00:19<01:14,  9.32s/it]

0.0035938136638 0.716316177719


 30%|███       | 3/10 [00:31<01:10, 10.11s/it]

...

7.74263682681 0.716396268218


 90%|█████████ | 9/10 [01:44<00:11, 11.96s/it]

27.8255940221 0.716433010332


100%|██████████| 10/10 [01:57<00:00, 12.29s/it]

100.0 0.716433957776
CPU times: user 1min 56s, sys: 15.7 s, total: 2min 12s
Wall time: 1min 58s


In [8]:
%%time
#попробуем найти более точное значение C
C_space = np.linspace(0.1, 1, 10)

for c in tqdm(C_space):
    clf = LogisticRegression(C=c)
    print(c, cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=kf).mean())

  0%|          | 0/10 [00:00<?, ?it/s] 10%|█         | 1/10 [00:12<01:50, 12.24s/it]

0.1 0.71640275448


 20%|██        | 2/10 [00:24<01:37, 12.17s/it]

0.2 0.716263802391


 30%|███       | 3/10 [00:36<01:25, 12.16s/it]

...

0.8 0.716150439988


 90%|█████████ | 9/10 [01:47<00:11, 11.98s/it]

0.9 0.716396289048


100%|██████████| 10/10 [02:04<00:00, 13.33s/it]

1.0 0.716348643726
CPU times: user 2min 2s, sys: 15.2 s, total: 2min 18s
Wall time: 2min 4s


В целом можно увидеть, что искать С более точно нет смысла. Причиной, почему наш скор так скачет, скорее всего является то, что не ни фиксировали разбиение(Random State) при кросс-валидации... Поэтому без зазрения совести можно взять C = 0.5.
<br>Соотвественно качество при нём примерно 0.716. Надо заметить, что оно площадь под рок-кривой получилась больше, чем при градиентном бустинге, хотя по идее деревья дольжны давать большую точность. Скорее всего это обусловлено тем, что в случае Градиентного бустинга мы просто недообучились и можно было увеличивать число деревьев(разница между 20 и 30 была порядка 0.01, что довольно много)
<br>И если говорить про скорость - регрессия безусловно быстрее

Окей, в прошлый раз мы поняли, что регрессия работает хорошо.. Однако с некоторыми переменнами мы поступали неправильно. Считали категориальные за числовые.

In [10]:
categorial_features = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                      'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
decimal_df = train_df.drop(categorial_features, axis=1)
X_decimal_train = StandardScaler().fit_transform(decimal_df)

Итак, мы выкинули категориальные признаки... Подбор С про обучении будет немного другим, чем в прошлый раз - отптимальное значение понятно, что лежит примерно от 0.1 до 10, да и точность не слишком важна

In [11]:
%%time
C_space = np.logspace(-2, 1, 10)

for c in tqdm(C_space):
    clf = LogisticRegression(C=c)
    print(c, cross_val_score(clf, X_decimal_train, y_train, scoring='roc_auc', cv=kf).mean())

  0%|          | 0/10 [00:00<?, ?it/s] 10%|█         | 1/10 [00:10<01:32, 10.25s/it]

0.01 0.716294508529


 20%|██        | 2/10 [00:22<01:25, 10.73s/it]

0.0215443469003 0.716612076807


 30%|███       | 3/10 [00:34<01:17, 11.12s/it]

...

2.15443469003 0.716775732218


 90%|█████████ | 9/10 [01:40<00:11, 11.05s/it]

4.64158883361 0.716403485595


100%|██████████| 10/10 [01:51<00:00, 11.06s/it]

10.0 0.716485323868
CPU times: user 1min 51s, sys: 15.1 s, total: 2min 6s
Wall time: 1min 51s


Как можно увидеть, качество практически не меняется - на четвёртый знак после запятой больше всего слияет random_seed. Причиной по которой такое может быть - наверное потому что в случае логистической регресии эти признаки слабо коррелировали с целевой переменной, по сути они были шумовые. Поэтому классификатор присвоил им малый вес - они сносили практически никакой вклад в ответ.

Кстати, про подсчёт героев... В pandas 0.20.0 будет функция nunique, применяемая ко всему датафрейму.
У нас есть есть пока только возможность работать с Серией.
Не совсем ясно про распределение героев по позициям, поэтому создадим супер-серию

In [19]:
hero_features = ['r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
hero_sr = train_df['r1_hero']
for label in hero_features:
    hero_sr.append(train_df[label])
print(hero_sr.nunique())
print(hero_sr.max())

108
112


Забавный момент... в обучающей выборке оказались не ве герои, из-за чегоя не смог воспользоваться мешком слов. Но выход легко нашёлся - можно было просто взять максимальный индекс героя.. И тогда он скорее всего совпадает с их количеством

Теперь надо добавить эти признаки обратно, вопспользуемся предложенным кодом

In [40]:
X_pick = np.zeros((train_df.shape[0], 112))

for i, match_id in enumerate(train_df.index):
    for p in range(5):
        X_pick[i, train_df.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, train_df.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
#преобразуем нашу матрицу к датафрейму и склеим
pick_df = pd.DataFrame(X_pick, index=train_df.index).fillna(0).astype('int')
full_train_df = decimal_df.join(pick_df)
full_train_df.head(5)

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,102,103,104,105,106,107,108,109,110,111
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0,0,1,0,0,0,0,0,0,0
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0,0,0,0,0,0,0,0,0,0
2,1430227081,4,1319,1270,22,0,0,12,3,1314,...,0,0,0,0,0,0,0,0,0,0
3,1430263531,4,1779,1056,14,0,0,5,2,539,...,0,0,0,0,0,0,0,0,0,0
4,1430282290,4,1431,1090,8,1,0,8,2,629,...,0,0,0,0,0,0,0,0,0,0


In [32]:
%%time
X_full_train = StandardScaler().fit_transform(full_train_df)

for c in tqdm(C_space):
    clf = LogisticRegression(C=c)
    print(c, cross_val_score(clf, X_full_train, y_train, scoring='roc_auc', cv=kf).mean())

  0%|          | 0/10 [00:00<?, ?it/s] 10%|█         | 1/10 [00:19<02:54, 19.38s/it]

0.01 0.7520283409


 20%|██        | 2/10 [00:41<02:40, 20.08s/it]

0.0215443469003 0.751742631573


 30%|███       | 3/10 [01:02<02:23, 20.45s/it]

...

2.15443469003 0.751781101435


 90%|█████████ | 9/10 [03:09<00:21, 21.20s/it]

4.64158883361 0.75193296372


100%|██████████| 10/10 [03:30<00:00, 21.08s/it]

10.0 0.7515363446
CPU times: user 3min 22s, sys: 23.6 s, total: 3min 46s
Wall time: 3min 31s


Как видно отсюда, у нас удалось улучшить качество на кросс валидации до 0.7519 Это произошло ввиду того, что признаки оказались сильными:) И теперь они не случайные, а коррелируют с целевой переменной

<b>Грустный финал</b>
<br><br> Как оказалось зря с самого начала я не вставил тестовую часть:) Придётся быстренько её видоизменить

In [47]:
test_df = test_df.drop(categorial_features, axis=1)
X_test_pick = np.zeros((test_df.shape[0], 112))

for i, match_id in enumerate(test_df.index):
    for p in range(5):
        X_test_pick[i, test_data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_test_pick[i, test_data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
#преобразуем нашу матрицу к датафрейму и склеим
pick_test_df = pd.DataFrame(X_test_pick, index=test_df.index).fillna(0).astype('int')
full_test_df = test_df.join(pick_test_df)
full_test_df.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,102,103,104,105,106,107,108,109,110,111
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,4,1103,1089,8,0,1,9,3,1183,...,0,0,0,0,0,0,0,0,0,0
7,1430293357,2,556,570,1,0,0,9,4,1194,...,0,0,0,0,0,0,0,0,0,0
10,1430301774,2,751,808,1,0,0,13,2,421,...,0,0,0,0,0,0,0,0,0,1
13,1430323933,3,708,903,1,1,1,11,2,672,...,0,0,0,0,0,0,0,-1,0,0
16,1430331112,4,1259,661,4,0,0,9,5,1703,...,0,0,0,0,0,0,0,0,0,0


In [53]:
X_full_test = StandardScaler().fit_transform(full_test_df)
clf = LogisticRegression(C=0.5)
clf.fit(X_full_train, y_train)
pred = clf.predict_proba(X_full_test)[:, 1]
pred[:5]

array([ 0.82494991,  0.75850176,  0.18702219,  0.85934659,  0.23762798])

In [54]:
sample_submission = pd.read_csv('data/sample_submission.csv', index_col='match_id')

In [55]:
sample_submission.head()

,radiant_win
match_id,
6,0.518503
7,0.518503
10,0.518503
13,0.518503
16,0.518503


In [57]:
submission = sample_submission.copy()
submission['radiant_win'] = pred
submission.to_csv('baseline.csv')

Дмитрий Торшин	0.75522	- Mon, 27 Mar 2017 11:54:49

In [58]:
print(pred.min(), pred.max())

0.00868961197807 0.996484254327
